In [ ]:
import pandas as pd
import gc
df_train =pd.read_csv('/home/kesci/input/bytedance/train_final.csv',names=['qId','q','aId','a','target'],nrows=50000000)
df_test =pd.read_csv('/home/kesci/input/bytedance/bytedance_contest.final_2.csv',names=['qId','q','aId','a'])
df_test['a']=df_test['a'].apply(lambda x:x[:-1])
gc.collect()

In [ ]:
def generate_qid_graph_table(row):
    hash_key1 = row["q"]
    hash_key2 = row["a"]

    qid_graph.setdefault(hash_key1, []).append(hash_key2)
    qid_graph.setdefault(hash_key2, []).append(hash_key1)

In [ ]:
qid_graph = {}
df_train.apply(generate_qid_graph_table, axis=1)
df_test.apply(generate_qid_graph_table, axis=1)

In [ ]:
def pagerank():
    MAX_ITER = 35
    d = 0.85

    pagerank_dict = {i: 1 / len(qid_graph) for i in qid_graph}
    num_nodes = len(pagerank_dict)

    for iter in range(0, MAX_ITER):
        print(iter)
        for node in qid_graph:
            local_pr = 0

            for neighbor in qid_graph[node]:
                local_pr += pagerank_dict[neighbor] / len(qid_graph[neighbor])

            pagerank_dict[node] = (1 - d) / num_nodes + d * local_pr

    return pagerank_dict

In [ ]:
pagerank_dict = pagerank()

In [ ]:
import pickle
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
save_obj(pagerank_dict,'pagerank')

In [ ]:
import pickle
import pandas as pd
import gc
def load_obj(name ):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)
pagerank_dict=load_obj('pagerank')

In [ ]:
def get_pagerank_fun(df):
    result=[]
    def get_pagerank_value(row):
        result.append([pagerank_dict[row["q"]],pagerank_dict[row["a"]]])
    print('start')
    pr_feats=df.apply(get_pagerank_value, axis=1)
    print('done')
    return pd.DataFrame(result)

In [ ]:
df_train =pd.read_csv('/home/kesci/input/bytedance/train_final.csv',names=['qId','q','aId','a','target'],nrows=50000000)
df_test =pd.read_csv('/home/kesci/input/bytedance/bytedance_contest.final_2.csv',names=['qId','q','aId','a'])
df_test['a']=df_test['a'].apply(lambda x:x[:-1])

In [ ]:
df1=df_train[['q','a']].iloc[0:10000000,]
df2=df_train[['q','a']].iloc[10000000:20000000,]
df3=df_train[['q','a']].iloc[20000000:30000000,]
df4=df_train[['q','a']].iloc[30000000:40000000,]
df5=df_train[['q','a']].iloc[40000000:50000000,]

df6=df_test[['q','a']].iloc[0:10000000,]
df7=df_test[['q','a']].iloc[10000000:20000000,]
df8=df_test[['q','a']].iloc[20000000:30000000,]
df9=df_test[['q','a']].iloc[30000000:40000000,]
df10=df_test[['q','a']].iloc[40000000:50000000,]
df11=df_test[['q','a']].iloc[50000000:60000000,]
df12=df_test[['q','a']].iloc[60000000:70000000,]
df13=df_test[['q','a']].iloc[70000000:80000000,]
df14=df_test[['q','a']].iloc[80000000:90000000,]
df15=df_test[['q','a']].iloc[90000000:,]

In [ ]:
del df_train,df_test
from multiprocessing import cpu_count
from multiprocessing import Pool
print(cpu_count())

In [ ]:
p= Pool(15)
df1=p.apply_async(get_pagerank_fun, args=(df1,))
df2=p.apply_async(get_pagerank_fun, args=(df2,))
df3=p.apply_async(get_pagerank_fun, args=(df3,))
df4=p.apply_async(get_pagerank_fun, args=(df4,))
df5=p.apply_async(get_pagerank_fun, args=(df5,))
df6=p.apply_async(get_pagerank_fun, args=(df6,))
df7=p.apply_async(get_pagerank_fun, args=(df7,))
df8=p.apply_async(get_pagerank_fun, args=(df8,))
df9=p.apply_async(get_pagerank_fun, args=(df9,))
df10=p.apply_async(get_pagerank_fun, args=(df10,))
df11=p.apply_async(get_pagerank_fun, args=(df11,))
df12=p.apply_async(get_pagerank_fun, args=(df12,))
df13=p.apply_async(get_pagerank_fun, args=(df13,))
df14=p.apply_async(get_pagerank_fun, args=(df14,))
df15=p.apply_async(get_pagerank_fun, args=(df15,))
p.close()
p.join()

In [ ]:
df1=df1.get()
df2=df2.get()
df3=df3.get()
df4=df4.get()
df5=df5.get()
df6=df6.get()
df7=df7.get()
df8=df8.get()
df9=df9.get()
df10=df10.get()
df11=df11.get()
df12=df12.get()
df13=df13.get()
df14=df14.get()
df15=df15.get()

In [ ]:
train_pr=pd.concat([df1,df2,df3,df4,df5])
test_pr=pd.concat([df6,df7,df8,df9,df10,df11,df12,df13,df14,df15])
test_pr.columns=['pr_q','pr_a']
train_pr.columns=['pr_q','pr_a']
test_pr=test_pr*100000000
train_pr=train_pr*100000000
train_pr.to_csv('/home/kesci/work/train_feature_pr_final.csv',index=False)
test_pr.to_csv('/home/kesci/work/test_feature_pr_final.csv',index=False)